# Example: Fit DMetrics

This notebook loads the bundled metrics data

In [1]:
import importlib.resources as resources

import pandas as pd
import numpy as np

import pysrat.nhpp as srm

from pysrat.data import NHPPData, DMetricsData
from pysrat.nhpp.multifactor import MFLogitNHPP, MFProbitNHPP, MFCloglogNHPP

In [2]:
data_path = resources.files("pysrat").joinpath("datasets/dmetrics/dmetrics1.csv")
df = pd.read_csv(data_path)
df

day  fault   tc  ctc    cov   ccov
0     1      3    5    5  0.018  0.018
1     2     16  175  180  0.409  0.427
2     3      9  186  366  0.066  0.493
3     4      3   67  433  0.132  0.625
4     5      2   14  447  0.013  0.638
5     6      0   54  501  0.055  0.693
6     7      2   28  529  0.021  0.714
7     8      1   15  544  0.018  0.732
8     9      1   23  567  0.024  0.756
9    10      2   17  584  0.026  0.782
10   11      9   74  658  0.051  0.833
11   12      3   43  701  0.011  0.844
12   13      2   20  721  0.002  0.846
13   14      1   12  733  0.013  0.859
14   15      2   18  751  0.018  0.877
15   16      7   19  770  0.018  0.895
16   17      3    9  779  0.000  0.895
17   18      0    9  788  0.003  0.898
18   19      0   32  820  0.014  0.912
19   20      0    8  828  0.000  0.912

In [3]:
data = DMetricsData.from_dataframe(df, fault='fault', metrics=df.columns.drop('fault').to_list())

In [4]:
models = {
    "logit": MFLogitNHPP(),
    "probit": MFProbitNHPP(),
    "cloglog": MFCloglogNHPP(),
}

for model in models.values():
    model.fit(data, verbose=True)

In [5]:
summary = []
for name, model in models.items():
    summary.append({
        "name": name,
        "params_": model.params_.copy(),
        "aic_": float(model.aic_),
    })

summary

[{'name': 'logit',
  'params_': array([ 6.60331317e+01, -3.67104467e+00,  5.22474322e-01,  1.77179246e-02,
         -6.34912977e-03, -4.57378492e-01, -4.23183288e-01]),
  'aic_': 82.89452065150758},
 {'name': 'probit',
  'params_': array([ 6.61238755e+01, -1.94479253e+00,  2.17540585e-01,  7.84299151e-03,
         -5.45702471e-04,  8.43977501e-01, -1.74933953e+00]),
  'aic_': 84.77255949217567},
 {'name': 'cloglog',
  'params_': array([ 6.60006949e+01, -3.72501426e+00,  5.52901578e-01,  1.81100945e-02,
         -9.24633065e-03, -1.70250033e+00,  1.44403488e+00]),
  'aic_': 81.4471627440704}]

In [7]:
models['logit']

MFLogitNHPP(omega=66.0331, intercept=-3.67104, day=0.522474, tc=0.0177179, ctc=-0.00634913, cov=-0.457378, ccov=-0.423183)

In [9]:
srm.plot_dmvf(NHPPData.from_intervals(counts=data.fault), models['logit'])

<Axes: xlabel='time', ylabel='# of faults'>

In [11]:
best = srm.stepwise(models['logit'], verbose=True)
best

step 1 add tc -> AIC=86.0209
step 2 add ctc -> AIC=79.0744


MFLogitNHPP(omega=66.0417, intercept=-3.78901, day=0.503211, tc=0.0167946, ctc=-0.00631687)

In [12]:
srm.plot_dmvf(NHPPData.from_intervals(counts=df['fault']), best)

<Axes: xlabel='time', ylabel='# of faults'>